# 1.1 (a)

In [37]:
# coding: utf-8
# SNLP - SoSe 2019 - ASSINGMENT VI

import math
import re
from collections import defaultdict, Counter
import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt 


def tokenize(text):
    "List all the word tokens (consecutive letters) in a text. Normalize to lowercase."
    return re.findall('[a-z]+', text.lower())

def word_ngrams(tokenized_text, n):
    return [tuple(tokenized_text[i:i + n]) for i in range(0, len(tokenized_text)-n+1)]


class ngram_LM:
    """A class to represent a language model."""

    def __init__(self, n, ngram_counts, vocab, unk=False):
        """"Make a n-gram language model, given a vocab and
            data structure for n-gram counts."""

        self.n = n

        self.vocab = vocab

        self.V = len(vocab)

        self.ngram_counts = ngram_counts

    # YOUR CODE HERE
    # START BY MAKING THE RIGHT COUNTS FOR THIS PARTICULAR self.n
        # for unigrams, we only need total word count
        if n == 1:
            self.total_count = sum(self.ngram_counts.values())
        # for bigrams, we need total count wrt each word. In our language, it is history count.
        elif n == 2:
            self.history_count = Counter()
            for k, v in self.ngram_counts.items():
                self.history_count[k[0]] = self.history_count[k[0]] + v
            # since we only count for the first word in the tuple, we will always
            # miss counting </s>. However, since the frequency of </s> is the same
            # as the frequency of <s>, we can simply assign it equal to it.
            self.history_count['</s>'] = self.history_count['<s>']



    def estimate_prob(self, history, word):
        """Estimate probability of a word given a history."""
        if history == '':
            # unigram
            word_frequency = self.ngram_counts[tuple([word])]
            return word_frequency/self.total_count

        else:
            # bigram
            word_frequency = self.ngram_counts[tuple([history, word])]
            history_count = self.history_count[history]
            if history_count == 0:
                return 0
            return word_frequency/history_count


    def estimate_smoothed_prob(self, history, word, alpha = 0.5):
        """Estimate probability of a word given a history with Lidstone smoothing."""

        if history == '':
            # unigram
            word_frequency = self.ngram_counts[tuple([word])]
            return (word_frequency + alpha)/(alpha*self.V +self.total_count)

        else:
            # bigram
            word_frequency = self.ngram_counts[tuple([history, word])]
            history_count = self.history_count[history]
            return (word_frequency + alpha)/(alpha*self.V + history_count)

    def getN1plus(self, word):
        # currently implemented for N1+(.,w)
        count = 0
        for v in self.vocab:
            if self.ngram_counts[tuple([v,word])] > 0:
                count = count + 1
        return count

    
    def kneser_Ney_smoothing(self, history, word):
        # currently implemented only for history = ''
        
        # out of vocab case
        if word not in self.vocab:
            return 1/self.V
        numerator = self.getN1plus(word)
        denominator = len(self.ngram_counts)
        return numerator/denominator
    


    def logP(self, history, word):
        """Return base-2 log probablity."""
        prob = self.estimate_smoothed_prob(history, word)
        log_prob = math.log(prob, 2)
        return log_prob


    def score_sentence(self, sentence):
        """Given a sentence, return score."""
        log_prob_sum = 0
        for i in range(len(sentence)):
            history = sentence[i][0]
            word = sentence[i][1]
            log_prob = self.logP(history, word)
            log_prob_sum += log_prob
        normalized_log_prob_sum = (-1 / len(sentence)) * log_prob_sum
        return normalized_log_prob_sum


    def test_LM(self):
        """Test whether or not the probability mass sums up to one."""

        precision = 10**-8

        if self.n == 1:

            P_sum = sum(self.estimate_prob('', w) for w in self.vocab)

            assert abs(1.0 - P_sum) < precision, 'Probability mass does not sum up to one.'

        elif self.n == 2:
            histories = ['the', 'in', 'at', 'blue', 'white']

            for h in histories:

                P_sum = sum(self.estimate_prob(h, w) for w in self.vocab)

                assert abs(1.0 - P_sum) < precision, 'Probability mass does not sum up to one for history' + h

        print('TEST SUCCESSFUL!')



    def test_smoohted_LM(self):
        """Test whether or not the smoothed probability mass sums up to one."""
        precision = 10**-8

        if self.n == 1:

            P_sum = sum(self.estimate_smoothed_prob('', w) for w in self.vocab)

            assert abs(1.0 - P_sum) < precision, 'Probability mass does not sum up to one.'

        elif self.n == 2:
            histories = ['the', 'in', 'at', 'blue', 'white']

            for h in histories:

                P_sum = sum(self.estimate_smoothed_prob(h, w) for w in self.vocab)

                assert abs(1.0 - P_sum) < precision, 'Probability mass does not sum up to one for history' + h

        print('TEST SUCCESSFUL!')


    def perplexity(self, test_corpus, alpha):

        likelihood = 0
        for sentence in test_corpus:
            try:
                if self.n == 1:
                    prob = self.estimate_smoothed_prob('', sentence[0], alpha)
                elif self.n ==2:
                    prob = self.estimate_smoothed_prob(sentence[0], sentence[1], alpha)
                likelihood += math.log2(prob)
            except:
                if alpha == 0:
                    continue 

        perplexity = math.pow(2, (-1*likelihood)/len(test_corpus))
        return perplexity


In [33]:
# main
filename= 'corpus.sent.en.train'
with open(filename, encoding='utf-8', errors='replace') as f:
    # read entire file
    text = f.read() 

# tokenize it
tokenized_text = tokenize(text)
unigrams = Counter(word_ngrams(tokenized_text,1))
unigram_vocabulary = list(unigrams.keys())
unigram_vocabulary = [i[0] for i in unigram_vocabulary]

bigrams = Counter(word_ngrams(tokenized_text,2))
bigram_vocabulary = [i for i in unigram_vocabulary]
bigram_vocabulary.extend(['<s>','</s>'])



In [38]:
words = ['york', 'matter']
unigram_LM = ngram_LM(1, unigrams, unigram_vocabulary)
bigram_LM = ngram_LM(2, bigrams, bigram_vocabulary)
for word in words:
    print('\n\n------ For Word "{}"------\n\n'.format(word))
    print('N(w) is {}'.format(unigram_LM.ngram_counts[tuple([word])]))
    print('N1+(.,w) is {}'.format(bigram_LM.getN1plus(word)))
    
    P_ML = unigram_LM.estimate_prob('',word)
    log2P_ML = -math.log(P_ML, 2)
    print('-log2P_ML(w) is {}'.format(log2P_ML))
    
    P_Lids = unigram_LM.estimate_smoothed_prob('',word,1)
    log2P_Lids =  -math.log(P_Lids, 2)
    print('-log2PLids(w) is {}'.format(log2P_Lids))
    
    P_kn = bigram_LM.kneser_Ney_smoothing('',word)
    log2P_kn = -math.log(P_kn, 2)
    print('-log2Pkn(w) is {}'.format(log2P_kn))






------ For Word "york"------


N(w) is 2374
N1+(.,w) is 9
-log2P_ML(w) is 12.258374283571047
-log2PLids(w) is 12.268770360977008
-log2Pkn(w) is 18.06307025575544


------ For Word "matter"------


N(w) is 2367
N1+(.,w) is 269
-log2P_ML(w) is 12.262634512478337
-log2PLids(w) is 12.27302879345373
-log2Pkn(w) is 13.16153289464113
